In [2]:
!pip install sentence_transformers

  Using cached PyYAML-6.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 3.8 MB/s eta 0:00:00a 0:00:01
Using cached PyYAML-6.0.2-cp313-cp313-macosx_11_0_arm64.whl (171 kB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 4.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12.1
    Uninstalling sympy-1.12.1:
      Successfully uninstalled sympy-1.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pennylane-qiskit 0.41.0.post0 requires qiskit<1.3,>=0.32, but you have qiskit 1.4.2 which is incompatible.
pennylane-qiskit 0.41.0.post0 requires sympy<1.13, but you have sympy 1.14.0 which is incompatible.


In [3]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from wordcloud import WordCloud
from sentence_transformers import SentenceTransformer

/Users/eric1026/qiskit/miniconda3/envs/qiskit/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# --- 1. 合併所有推文資料 ---
csv_files = sorted(glob.glob("data/tariff_data_en/*.csv"))
df_list = [pd.read_csv(f) for f in csv_files]
df = pd.concat(df_list, ignore_index=True)
df.head()

,Timestamp,Tweet Content,sentiment
0,2025-03-01T22:17:49.000Z,"Proposed tariffs on China, Canada, and Mexico ...",positive
1,2025-03-01T22:27:25.000Z,President Trump says his 25-percent tariff on ...,positive
2,2025-03-01T23:00:01.000Z,"""You should pay less for someone that has tari...",negative
3,2025-03-01T23:00:01.000Z,"""You should pay less for someone that has tari...",negative
4,2025-03-01T22:30:20.000Z,Beer Companies Say Trump’s Aluminum Tariff Wil...,negative


In [5]:
# --- 2. 標準化欄位與時間處理 ---
df.columns = ['Timestamp', 'text', 'sentiment']
df.dropna(subset=['text'], inplace=True)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # 含 UTC 時區
event_date = pd.to_datetime("2025-04-02").tz_localize("UTC")
df['period'] = df['Timestamp'].apply(lambda x: 'before' if x < event_date else 'after')

In [6]:
# --- 3. 使用預訓練 Sentence-BERT 將推文轉成語意向量 ---
model = SentenceTransformer('all-MiniLM-L6-v2')  # 輕量快速
X = model.encode(df['text'].tolist(), show_progress_bar=True)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Batches: 100%|██████████| 280/280 [00:21<00:00, 13.24it/s]


In [7]:
# --- 4. KMeans 聚類 ---
k = 6
kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
# --- 5. PCA 降維 + 畫群體分布圖 ---
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

os.makedirs("saved_imgs", exist_ok=True)

plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['cluster'], cmap='tab10', alpha=0.6)
plt.title("KMeans with SBERT Embedding (PCA)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.colorbar(scatter, label='Cluster')
plt.savefig("saved_imgs/kmeans_pca_sbert.png")
plt.close()

In [11]:
# --- 6. 產出每個群的 WordCloud（照舊）---
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
custom_stopwords = set(['tariff', 'trump'])
all_stopwords = list(ENGLISH_STOP_WORDS.union(custom_stopwords))

def plot_cluster_wordcloud(df, cluster_num):
    text = " ".join(df[df['cluster'] == cluster_num]['text'])
    wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=all_stopwords).generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f"Cluster {cluster_num} WordCloud")
    plt.savefig(f"saved_imgs/cluster_{cluster_num}_wordcloud_sbert.png")
    plt.close()

for cl in sorted(df['cluster'].unique()):
    plot_cluster_wordcloud(df, cl)

### 🧠 群體語意總覽分析

本研究以 Sentence-BERT 模型將推文轉換為語意向量，並透過 KMeans 聚類方法將語料分為六個主題群。經由每群高頻詞彙的詞雲（word cloud）視覺化與語意對照分析後，歸納出各群的核心語境與主題如下：

- **Cluster 0 – 美加貿易爭端與加拿大觀點**  
  該群體聚焦於加拿大與美國之間的貿易關係，頻繁出現的詞彙包含 `Canada`, `Trudeau`, `threat`, `tariffs`，顯示該群多為對美方政策持批評態度的推文，語氣偏向防禦與不滿。

- **Cluster 1 – 市場波動與投資反應**  
  此群包含大量與金融市場相關語彙，如 `market`, `stock`, `investor`, `recession`，反映出用戶對於貿易政策可能對經濟與股市造成影響的擔憂與分析，語氣呈現理性但偏焦慮。

- **Cluster 2 – 中美貿易戰與國際談判**  
  詞彙如 `China`, `export`, `deal`, `war` 明確指出該群主題為中美貿易衝突與談判進展，語氣多帶有對抗性或批判性，且事件後大量湧現，為本研究中最受事件影響的群體。

- **Cluster 3 – 關稅影響與消費經濟**  
  該群多出現 `cost`, `import`, `consumer`, `price`，代表對關稅政策對商品價格與消費者負擔的關注。語氣較為理性、經濟分析導向，屬於討論型內容。

- **Cluster 4 – 政策評論與官方言論反應**  
  含有 `President`, `policy`, `economy`, `administration` 等詞彙，代表該群集中在政府官員與政策制定者的發言評論，語氣可能因政治立場不同而分化，整體呈現評論性與新聞轉述性質。

- **Cluster 5 – 網民輿論與口語表態**  
  本群聚焦於網路社群中的一般輿論，詞彙如 `just`, `say`, `like`, `want` 表現出明顯的非正式、情緒化用語，語氣較為自由、多樣，並包含支持、諷刺與戲謔等多元觀點，是情緒最偏向正面的群體。

整體而言，分群結果有效劃分了不同語意主題，並揭示了政策事件如何在語意與情緒層面觸發特定群體的反應，為理解社群輿


### 📋 群體語意總覽表格

| Cluster | 主題焦點                         | 高頻關鍵詞（部分）                           | 語氣傾向         | 備註                             |
|---------|----------------------------------|----------------------------------------------|------------------|----------------------------------|
| 0       | 美加貿易爭端與加拿大觀點         | Canada, Trudeau, threat, tariffs, US         | 負面、批判       | 舊議題為主，事件前集中出現       |
| 1       | 市場波動與投資反應               | market, stock, investor, recession, risk     | 焦慮、理性       | 與金融市場連動，反應全球不安     |
| 2       | 中美貿易戰與國際談判             | China, deal, war, export, negotiation        | 對立、憤怒       | 事件後激增，情緒最負面           |
| 3       | 關稅影響與消費經濟               | cost, import, consumer, price, impact        | 中性、理性       | 關注稅收與價格影響               |
| 4       | 政策評論與官方言論反應           | President, policy, administration, economy   | 評論、兩極       | 涉及領導人與政府發言             |
| 5       | 網民輿論與口語表態               | just, like, say, want, week, new             | 多元、偏正向     | 社群用語多，情緒最活躍           |


In [10]:
# --- 7. 額外分析輸出：事件前後各群數量與情緒分布 ---
print("\n📊 各群情緒比例（row-normalized）:")
print(df.groupby('cluster')['sentiment'].value_counts(normalize=True).unstack().fillna(0))

print("\n📆 各群在事件前/後的比例（row-normalized）:")
print(pd.crosstab(df['cluster'], df['period'], normalize='index'))


📊 各群情緒比例（row-normalized）:
sentiment  negative   neutral  positive
cluster                                
0          0.501894  0.174242  0.323864
1          0.536533  0.127507  0.335960
2          0.549412  0.164706  0.285882
3          0.380933  0.251810  0.367257
4          0.504292  0.194421  0.301288
5          0.425767  0.164417  0.409816

📆 各群在事件前/後的比例（row-normalized）:
period      after    before
cluster                    
0        0.089015  0.910985
1        0.393266  0.606734
2        0.755294  0.244706
3        0.446903  0.553097
4        0.443348  0.556652
5        0.461350  0.538650


## 分群情緒傾向與事件前後分布分析 （GPT）

為探討推文內容在語意與情緒上的異質性，我們使用 Sentence-BERT 嵌入向量與 KMeans 聚類對語料進行分群（共六群），並進一步分析各群的情緒比例及其在關稅政策事件（2025 年 4 月 2 日）前後的出現頻率。下列為各群重點分析結果：

1. 情緒傾向分析
透過原始資料中的情緒標註（positive、neutral、negative），我們計算了各群體內部的情緒分布。結果顯示，群 0、1、2 的負面情緒比例皆超過 50%，反映其內容多涉及爭議性話題或批判性語言。特別是群 2，負面情緒高達 54.9%，其主題以中美貿易戰為主，詞彙如 "Chinese", "war", "export", "negotiation" 明顯帶有強烈對立語意。

相較之下，群 3 與群 5 呈現相對中性或偏向正向情緒，其中群 5 的正向情緒比例最高（約 40.9%），顯示該群文本多以網民口語評論為主，語氣較為自由且表達支持、讚許等情緒的比例較高。群 3 則多與消費者成本、進口商品相關，語氣相對理性與分散。

2. 事件前後出現比例分析
對於事件時間（2025-04-02）前後的分布，我們發現各群反應明顯不同。群 2 的推文有超過 75% 發生於事件之後，顯示該主題與該事件（如新一輪關稅政策、重大外交爭端）有高度相關性，並引發大量後續討論。相對而言，群 0 的推文 91% 發生在事件之前，推測與早期美加貿易爭端相關的議題熱度已逐漸退燒。

其他群（群 1、3、4、5）則呈現出較為平均的分布，顯示這些主題在事件前後均有持續性討論。例如群 1（市場波動與投資）、群 4（政府政策與總統發言）皆為長期關注議題，並非僅由特定事件引發。

3. 總結
綜合語意與時間分析可知，本次事件最直接影響的群體為 Cluster 2，其不僅情緒傾向偏負面，亦在事件後大量增加。此結果突顯語意分群能有效區分出對政策事件敏感的主題群，為輿情變化監控與情緒預測提供依據。未來可考慮進一步追蹤此類事件引發的次波輿論走向，並結合情緒強度與主題動態模型以強化分析深度。

